In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(filename)s:%(message)s')  

In [2]:
# import all model concepts
from src.model import *

# import the simulator
from src.simulator.simulator import Simulator

# import a library that can visualise our systems
from src.ui import elk

In [3]:
class Resources(object):
    on_off = Resource("switch", ["on", "off"])
    flow = Resource("L/min", REAL)
    water = Resource("Liters", REAL)

In [5]:
class Pump(Entity):
    """
    A pump that is placed in an assumingly infinite source of water 
    (like a big water tank or lake), 
    we therefore don't model the pump's water input.
    
    For this example we also disregard the electricity that is required for the pump to operate.
    """
    switch = Input(resource=Resources.on_off, value="on")
    output = Output(resource=Resources.flow, value=0)
    
    on = State()
    off = current = State()
    
    # transitions and guards (as lambdas)
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.switch.value == "on"))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.switch.value == "off"))
    
    # update functions are related to a state and define the value of a port
    @update(state=on, target=output)
    def set_on(self, dt=0):
        return 1.5

    @update(state=off, target=output)
    def set_off(self, dt=0):
        return 0

elk.plot(Pump())

In [13]:
class Tank(Entity):
    inflow = Input(resource=Resources.flow, value=1)  # the current inflow
    outflow = Output(resource=Resources.flow, value=0)  # the current outflow
    spill = Output(resource=Resources.water, value=0)  # the amount of water spilt
    quantity = Output(resource=Resources.water, value=0)  # the volume of water that left the tank
    
    maxoutflow = Local(resource=Resources.flow, value=0.05) # the maximum outflow
    volume = Local(resource=Resources.water, value=1.5)  # the currently held water
    capacity = Local(resource=Resources.water, value=1.5)  # the maximum capacity
        
    """ States """
    full = State()
    partially = current = State()
    empty = State()
        
    """ Transitions """
    @transition(source=partially, target=full)
    def to_full(self):
        return self.volume.value >= self.capacity.value
    
    @transition(source=partially, target=empty)
    def to_empty(self):
        return self.volume.value <= 0
    
    @transition(source=[empty,full], target=partially)
    def to_partially(self):
        return 0 < self.volume.value and self.volume.value < self.capacity.value
    
    """ Updates """
    @update(state=[empty,partially,full], target=volume)
    def set_volume(self, dt):
        netflow = self.inflow.value - self.outflow.value
        theoretical = self.volume.value + netflow * dt
        if theoretical >= self.capacity.value:
            return self.capacity.value
        else:
            return theoretical
        
    @update(state=[partially,full], target=outflow)
    def set_outflow_nonempty(self, dt):
        return self.maxoutflow.value
    
    @update(state=empty, target=outflow)
    def set_outflow_empty(self, dt):
        if self.inflow.value > self.maxoutflow.value:
            return self.maxoutflow.value
        else:
            return self.inflow.value
        
    @update(state=[empty,partially,full], target=quantity)
    def increase_quantity(self, dt):
        return self.quantity.value + self.outflow.value * dt
    
    @update(state=full, target=spill)
    def increase_spill(self, dt):
        netflow = self.inflow.value - self.outflow.value
        if netflow > 0:
            return self.spill.value + netflow * dt
        else:
            return self.spill.value
    
elk.plot(Tank())    

In [12]:
class System(Entity):
    
    """ Ports """
    switch = Input(resource=Resources.on_off, value="on")
    output = Output(resource=Resources.flow, value=0)
    
    """ Subentities """
    pump = Pump()
    tank = Tank()
    
    """ State (no transitions) """
    state = current = State()
    
    """ Influences """
    connect_switch = Influence(source=switch, target=pump.switch)
    connect_output = Influence(source=tank.outflow, target=output)
    
sim = Simulator(System())
sim.stabilise()
sim.advance(1)
sim.advance(1.5)
sim.plot()

INFO:simulator.py:Time: 0 | Advancing 0 and stabilising system
INFO:simulator.py:Time: 0 | Port value changed: outflow (tank) 0 -> 0.05
INFO:basesimulator.py:Time: 0 | Firing transition <<to_full>> in tank (Tank) : partially -> full  | current global time: 0
INFO:simulator.py:Time: 0 | Port value changed: output () 0 -> 0.05
INFO:basesimulator.py:Time: 0 | Firing transition <<off_to_on>> in pump (Pump) : off -> on  | current global time: 0
INFO:simulator.py:Time: 0 | Port value changed: output (pump) 0 -> 1.5
INFO:simulator.py:Time: 0 | Received instructions to advance 1 time steps. (Current global time: 0)
INFO:basesimulator.py:There is no behaviour change reachable by time advance.
INFO:simulator.py:Time: 0 | No next transition
INFO:simulator.py:Time: 0 | Advancing 1
INFO:simulator.py:Time: 1 | Advancing 1 and stabilising system
INFO:simulator.py:Time: 1 | Port value changed: spill (tank) 0 -> 0.95
INFO:simulator.py:Time: 1 | Finished advance and update of system
INFO:simulator.py:Ti